<a href="https://colab.research.google.com/github/arielle00/CMPE257-Fall23-Arielle-Shnayder/blob/main/HW4_Task4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [123]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import pandas as pd
import random


train_data = np.loadtxt("ZipDigits.train",delimiter=" ",usecols=range(0, 257))
test_data = np.loadtxt("ZipDigits.test",delimiter=" ",usecols=range(0, 257))

y_train = np.where(train_data[:, 0] == 1, 1, -1)
y_test = np.where(test_data[:, 0] == 1, 1, -1)

def calculate_intensity(image):
    return np.mean(image[1:])

def calculate_symmetry(image):
    image = image[1:]
    left = image[:int(256/2)]
    right = image[int(256/2):]
    symmetry = np.mean(np.abs(left - np.flip(right)))
    return symmetry

train_features = np.array([[calculate_intensity(x), calculate_symmetry(x)] for x in train_data])
test_features = np.array([[calculate_intensity(x), calculate_symmetry(x)] for x in test_data])

In [124]:
df_train = pd.DataFrame(np.hstack([np.ones((train_features.shape[0], 1)), train_features, y_train.reshape(-1, 1)]),
                        columns=['x0', 'x1', 'x2', 'x3'])

df_test = pd.DataFrame(np.hstack([np.ones((test_features.shape[0], 1)), test_features, y_test.reshape(-1, 1)]),
                       columns=['x0', 'x1', 'x2', 'x3'])

def check_for_misclassified(w):
  misclassified = []
  w = np.array(w)
  signs = np.sign(np.dot(df_train.iloc[:, :3].values, w))
  y = df_train.iloc[:, 3].values

  mismatches = np.where(signs != y)[0]
  return mismatches


Ein_pla=[]
def perceptron():
  w = [0,0,0]
  misclassified = check_for_misclassified(w)
  i=0

  while(len(misclassified)!=0 and i<1000):
    Ein_pla.append(len(misclassified)/len(df_train))
    list_of_misclass = random.choice(list(misclassified))
    x = df_train.iloc[list_of_misclass][:3].values
    y = df_train.iloc[list_of_misclass][3]

    w = w + x*y

    misclassified = check_for_misclassified(w)
    i=i+1
  print("PLA")
  print("Ein:",np.mean(Ein_pla))
  print("This is the final weight: ",w)
  # plot_graph(w, "PLA")
  return w

w = perceptron()

X_test = df_test.iloc[:, :-1].values  # Extract input features
y_test = df_test.iloc[:, -1].values   # Extract true labels

y_pred = np.sign(np.dot(X_test,w))

error_rate_test = np.mean(y_pred != y_test)
print(f"Error rate on test set: {error_rate_test:.4f}")

PLA
Ein: 0.05614387601152106
This is the final weight:  [ 0.         -2.83613281 -9.69589063]
Error rate on test set: 0.0254


In [125]:
def pocketAlgo(misclassified_method, w_pocket):
  i=0
  Ein_list = []
  w_maybeNew = w_pocket
  misclassified_pocket = misclassified_method(w_pocket)

  while i<1000:
      list_of_misclass = random.choice(list(misclassified_pocket))
      Ein = len(misclassified_pocket)/len(df_train)
      Ein_list.append(Ein)

      x = df_train.iloc[list_of_misclass][:3].values
      y = df_train.iloc[list_of_misclass][3]

      w_maybeNew = w_pocket + x*y

      misclassified_new = misclassified_method(w_maybeNew)

      if len(misclassified_new) <= len(misclassified_pocket):
          w_pocket = w_maybeNew
          misclassified_pocket = misclassified_new
      else:
        misclassified_pocket = misclassified_method(w_pocket)
      i=i+1

  print("POCKET Ein",np.mean(Ein_list))
  print("This is the final weight: ",w_pocket)
  # plot_graph(w_pocket, "Pocket Algo")
  return w_pocket

w_pocket = [0,0,0]
w_pocket = pocketAlgo(check_for_misclassified, w_pocket)


X_test = df_test.iloc[:, :-1].values  # Extract input features
y_test = df_test.iloc[:, -1].values   # Extract true labels

y_pred = np.sign(np.dot(X_test,w_pocket))

error_rate_test = np.mean(y_pred != y_test)
print(f"Error rate on test set: {error_rate_test:.4f}")


POCKET Ein 0.020301193251954466
This is the final weight:  [ 0.         -0.16366797 -0.54210156]
Error rate on test set: 0.0244


In [126]:
y = list(df_train.iloc[:,-1])
df_train = df_train.iloc[:,:-1]
df_train = df_train.drop(columns="x0")

data_x = np.array(df_train)
data_poly = np.c_[[1]*len(data_x), data_x, data_x[:, 0]**2, data_x[:, 1]**2, data_x[:, 0] * data_x[:, 1], data_x[:, 0]**3, data_x[:, 1]**3, data_x[:, 0]**2*data_x[:, 1], data_x[:, 1]**2*data_x[:, 0]]
data_3d = pd.DataFrame(data_poly)
data_3d.insert(data_3d.shape[1],'y',y)

def check_missing_points_3d(w):
    X = data_3d.values
    hypo_y = np.sign(np.dot(X[:,:-1], w))
    mismatched_indices = np.where(hypo_y != data_3d["y"].values)[0]
    return mismatched_indices.tolist()

def pocket_algo_3d(w):
    global max_itr
    w_best = tuple(w)
    miss_points = tuple(check_missing_points_3d(w_best))
    w_best_points = len(miss_points)
    l_data = len(data_3d)
    Ein = []
    n = max_itr
    out_itr = n
    for i in range(n):
        j = miss_points[0]

        x = data_3d.iloc[j,:-1]
        y = data_3d.iloc[j,-1]
        w = w + x*y

        miss_points = tuple(check_missing_points_3d(w))
        w_points = len(miss_points)
        Ein.append(w_best_points/l_data)

        if(w_points < w_best_points):
            w_best = tuple(w)
            w_best_points = w_points
            out_itr = i

        if(w_points == 0):
            w_best = tuple(w)
            w_best_points = w_points
            break

    print("Final Weights (Pocket):",w_best, "Number of Missclassified points:", w_best_points, ", Got Final Weights at iteration", out_itr)
    print("In sample Error:", w_best_points/len(data_3d)*100,"%")
    return w_best

max_itr = 1000
w = [0]*10
w_pocket_3d = pocket_algo_3d(w)

Final Weights (Pocket): (-2.0, -2.205796875000006, -5.57365625000001, 3.6865865001220586, -3.846300031982417, 2.685121539001464, -3.9392629124803307, -2.254653077740262, -1.239758321208129, 2.105545955607579) Number of Missclassified points: 203 , Got Final Weights at iteration 123
In sample Error: 2.784254560416952 %


In [127]:
y = list(df_test.iloc[:,-1])

df_test = df_test.iloc[:,:-1]
df_test = df_test.drop(columns="x0")

data_x = np.array(df_test)
data_poly = np.c_[[1]*len(data_x), data_x, data_x[:, 0]**2, data_x[:, 1]**2, data_x[:, 0] * data_x[:, 1], data_x[:, 0]**3, data_x[:, 1]**3, data_x[:, 0]**2*data_x[:, 1], data_x[:, 1]**2*data_x[:, 0]]
data_3d = pd.DataFrame(data_poly)
data_3d.insert(data_3d.shape[1],'y',y)

miss_points = check_missing_points_3d(w_pocket_3d)
ein_pocket_test = len(miss_points)/len(data)*100
print("Test data in Sample Error is", ein_pocket_test, "%")
print("Number of misclassified points:", len(miss_points))

Test data in Sample Error is 2.8400597907324365 %
Number of misclassified points: 57
